In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
import requests
from io import BytesIO
import pygris

Process the raw excel file

In [2]:
# Read the excel sheet and skip blank rows
raw_excel = pd.read_excel("data/DSHA LIHTC List_MAPPING.xlsx", engine='openpyxl', skiprows=[2,3], skipfooter=4, dtype=str)
# Add additional column information from the first row
raw_excel.columns = (raw_excel.columns.astype(str) + " " + raw_excel.head(1).fillna("").astype(str)).iloc[0].str.strip().values
raw_excel.rename(columns={"ALLOCATION .1 DATE": "ALLOCATION DATE", "ALLOCATION  AMOUNT": "ALLOCATION AMOUNT", "Type of Property*": "Type of Property"}, inplace=True)
raw_excel.drop(0, inplace=True)

In [3]:
# Find and label the rows about tax year
raw_excel["is tax"] = raw_excel["PROJECT NAME & ADDRESS"].str.contains("TAX CREDIT ALLOCATIONS")

# Function that maps the boolean column "is tax", which is true when a row contains tax year information, to an integer equal to the tax year
def assign_to_year(x, i):
    # if the row is a tax year, increment i
    if x:
        i[0] = i[0] + 1
    # return an integer equal to the tax year
    return i[0] + 2016

# add a column for the tax year
index = [-1]
raw_excel["Tax Allocation Year"] = raw_excel["is tax"].apply(assign_to_year, args=[index])

# drop rows of tax year information and reformat
raw_excel = raw_excel.loc[~raw_excel["is tax"]].drop(columns="is tax").reset_index(drop=True)


In [4]:
# Add a column that labels the primary line for an entry
raw_excel["primary"] = ~raw_excel["County"].isna()

# Function that maps each the boolean column "primary", which is true when a row contains the primary info from the dataset, to an integer that functions as an index for primary entries
def assign_to_year(x, i):
    # if the row is primary, increment i
    if x:
        i[0] = i[0] + 1
    # return an index for the primary entries
    return i[0]

# add an index column for the primary entries
index = [-1]
raw_excel["primary"] = raw_excel["primary"].apply(assign_to_year, args=[index])

In [5]:
# Fix 'ALLOCATION AMOUNT', 'ALLOCATION DATE' swap
flipped_years = [2018, 2019, 2020, 2021, 2022]
tmp = raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years)]['ALLOCATION AMOUNT'].copy()
tmp2 = raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years)]['ALLOCATION DATE'].copy()
raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years), 'ALLOCATION AMOUNT'] = tmp2.values
raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years), 'ALLOCATION DATE'] = tmp.values


In [6]:
# Create separate dataframes for each row in a data entry
grouped_data = raw_excel.groupby("primary")
raw_data1 = grouped_data.nth(0)
raw_data2 = grouped_data.nth(1).drop(columns="Tax Allocation Year")
raw_data3 = grouped_data.nth(2).drop(columns="Tax Allocation Year")
raw_data4 = grouped_data.nth(3).drop(columns="Tax Allocation Year")
raw_data5 = grouped_data.nth(4).drop(columns="Tax Allocation Year")

# Modify the column names for each dataframe to prepare for joining
raw_data2.columns = raw_data2.columns + " 2"
raw_data3.columns = raw_data3.columns + " 3"
raw_data4.columns = raw_data4.columns + " 4"
raw_data5.columns = raw_data5.columns + " 5"

# Join the dataframes by index and remove unused columns
flattened_data = raw_data1.join(raw_data2, how="left").join(raw_data3, how="left").join(raw_data4, how="left").join(raw_data5, how="left").dropna(axis=1, how='all').reset_index(drop=True)


In [7]:
# Convert dates back to their orginial format
flattened_data["Placed in Service Date"] = pd.to_datetime(flattened_data["Placed in Service Date"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["ALLOCATION DATE"] = pd.to_datetime(flattened_data["ALLOCATION DATE"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Tax Credit Compliance Date"] = pd.to_datetime(flattened_data["Tax Credit Compliance Date"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Extended Use Period"] = pd.to_datetime(flattened_data["Extended Use Period"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Placed in Service Date 2"] = pd.to_datetime(flattened_data["Placed in Service Date 2"], errors='coerce').dt.strftime('%m/%d/%Y')



In [8]:
flattened_data


,PROJECT NAME & ADDRESS,ALLOCATION AMOUNT,ALLOCATION DATE,Placed in Service Date,4% or 9% Allocation,Applicable Credit Rate,Status Active/Non,# of Tax Credit Units,Type of Property,County,Tax Credit Compliance Date,Extended Use Period,Tax Allocation Year,PROJECT NAME & ADDRESS 2,ALLOCATION AMOUNT 2,Placed in Service Date 2,Applicable Credit Rate 2,PROJECT NAME & ADDRESS 3,PROJECT NAME & ADDRESS 4,PROJECT NAME & ADDRESS 5
0,PEARL CENTER,913337,12/20/2016,08/31/2018,0.09,9,Active,51,F,New Castle,12/31/2032,12/31/2048,2016,300 East 8th Street,NaN,NaN,NaN,"Wilmington, DE 19801",NaN,NaN
1,JEFFERSON ESTATES II,518044,12/20/2016,01/23/2018,0.09,9,Active,32,F,Sussex,12/31/2006,12/31/2021,2016,"825 Kings Highway, #101",NaN,NaN,NaN,"Lewes, DE 19958",NaN,NaN
2,VILLAGE OF IRON BRANCH (fka Halls Heritage),792137,12/20/2016,12/31/2018,0.09,9,Active,38,F,Sussex,NaN,NaN,2016,Halls Heritage Circle,NaN,NaN,NaN,"Millsboro, DE 19966",NaN,NaN
3,"THE FLATS, PHASE II MRB",503861,12/08/2016,06/28/2018,0.04,3.21,Active,72,F,New Castle,12/31/2033,12/31/2048,2016,Bancroft Parkway,NaN,NaN,NaN,"Wilmington, DE 19805",NaN,NaN
4,LIBERTY COURT MRB,620003,07/26/2017,10/13/2018,0.04,3.22,Active,100,F,Kent,12/31/2033,12/31/2048,2017,1289 Walker Road,NaN,NaN,NaN,"Dover, DE 19901",NaN,NaN
5,H. FLETCHER BROWN MRB,565754,05/12/2017,12/31/2019,0.04,3.24,Active,35,S,New Caslte,NaN,NaN,2017,1010 N. Broom Street,NaN,NaN,NaN,"Wilmington, DE 19806",NaN,NaN
6,CHELTEN APARTMENTS MRB,497801,08/25/2017,10/31/2018,0.04,3.22,Active,120,S,New Castle,12/31/2032,12/31/2048,2017,431 Old Forge Road,NaN,NaN,NaN,"New Castle, DE 19720",NaN,NaN
7,VILLAGE OF ST. JOHN (Forward Reservation),778622,12/06/2017,08/26/2019,0.09,9,Carryover,53,S,New Castle,NaN,NaN,2017,2019 North Market Street,NaN,NaN,NaN,"Wilmington, DE 19802",NaN,NaN
8,OUR LADY OF GRACE (Forward Reservation),774946,12/11/2017,12/20/2019,0.09,9,Carryover,60,F,New Castle,NaN,NaN,2017,2000 Mary Anagela Way,NaN,NaN,NaN,"Newark, DE 19712",NaN,NaN
9,BRANDYWINE STATION,1044577,12/11/2017,12/06/2018,0.09,9,Carryover,56,F,Sussex,NaN,NaN,2017,Case Lane and Edwards Boulevard,NaN,12/20/2019,NaN,"Millsboro, DE 19966",NaN,NaN


In [9]:
# Combine address fields
address_columns = ["PROJECT NAME & ADDRESS", "PROJECT NAME & ADDRESS 2", "PROJECT NAME & ADDRESS 3", "PROJECT NAME & ADDRESS 4", "PROJECT NAME & ADDRESS 5"]

# Extracts addresses from projects
def extract_address(x):
    # Project 27 has three full addresses, so we use the last one listed
    if x.name == 27:
        addr = x[address_columns].dropna().values[-1]
        return addr
    # The last two lines of the address field contain the address split between two lines, except for project 27
    else:
        addr = x[address_columns].dropna().values[-2:]
        return addr[0] + ", " + addr[1]

# Extract an address for each project
flattened_data["address"] = flattened_data.apply(extract_address, axis=1)

In [10]:
flattened_data

,PROJECT NAME & ADDRESS,ALLOCATION AMOUNT,ALLOCATION DATE,Placed in Service Date,4% or 9% Allocation,Applicable Credit Rate,Status Active/Non,# of Tax Credit Units,Type of Property,County,...,Extended Use Period,Tax Allocation Year,PROJECT NAME & ADDRESS 2,ALLOCATION AMOUNT 2,Placed in Service Date 2,Applicable Credit Rate 2,PROJECT NAME & ADDRESS 3,PROJECT NAME & ADDRESS 4,PROJECT NAME & ADDRESS 5,address
0,PEARL CENTER,913337,12/20/2016,08/31/2018,0.09,9,Active,51,F,New Castle,...,12/31/2048,2016,300 East 8th Street,NaN,NaN,NaN,"Wilmington, DE 19801",NaN,NaN,"300 East 8th Street, Wilmington, DE 19801"
1,JEFFERSON ESTATES II,518044,12/20/2016,01/23/2018,0.09,9,Active,32,F,Sussex,...,12/31/2021,2016,"825 Kings Highway, #101",NaN,NaN,NaN,"Lewes, DE 19958",NaN,NaN,"825 Kings Highway, #101, Lewes, DE 19958"
2,VILLAGE OF IRON BRANCH (fka Halls Heritage),792137,12/20/2016,12/31/2018,0.09,9,Active,38,F,Sussex,...,NaN,2016,Halls Heritage Circle,NaN,NaN,NaN,"Millsboro, DE 19966",NaN,NaN,"Halls Heritage Circle, Millsboro, DE 19966"
3,"THE FLATS, PHASE II MRB",503861,12/08/2016,06/28/2018,0.04,3.21,Active,72,F,New Castle,...,12/31/2048,2016,Bancroft Parkway,NaN,NaN,NaN,"Wilmington, DE 19805",NaN,NaN,"Bancroft Parkway, Wilmington, DE 19805"
4,LIBERTY COURT MRB,620003,07/26/2017,10/13/2018,0.04,3.22,Active,100,F,Kent,...,12/31/2048,2017,1289 Walker Road,NaN,NaN,NaN,"Dover, DE 19901",NaN,NaN,"1289 Walker Road, Dover, DE 19901"
5,H. FLETCHER BROWN MRB,565754,05/12/2017,12/31/2019,0.04,3.24,Active,35,S,New Caslte,...,NaN,2017,1010 N. Broom Street,NaN,NaN,NaN,"Wilmington, DE 19806",NaN,NaN,"1010 N. Broom Street, Wilmington, DE 19806"
6,CHELTEN APARTMENTS MRB,497801,08/25/2017,10/31/2018,0.04,3.22,Active,120,S,New Castle,...,12/31/2048,2017,431 Old Forge Road,NaN,NaN,NaN,"New Castle, DE 19720",NaN,NaN,"431 Old Forge Road, New Castle, DE 19720"
7,VILLAGE OF ST. JOHN (Forward Reservation),778622,12/06/2017,08/26/2019,0.09,9,Carryover,53,S,New Castle,...,NaN,2017,2019 North Market Street,NaN,NaN,NaN,"Wilmington, DE 19802",NaN,NaN,"2019 North Market Street, Wilmington, DE 19802"
8,OUR LADY OF GRACE (Forward Reservation),774946,12/11/2017,12/20/2019,0.09,9,Carryover,60,F,New Castle,...,NaN,2017,2000 Mary Anagela Way,NaN,NaN,NaN,"Newark, DE 19712",NaN,NaN,"2000 Mary Anagela Way, Newark, DE 19712"
9,BRANDYWINE STATION,1044577,12/11/2017,12/06/2018,0.09,9,Carryover,56,F,Sussex,...,NaN,2017,Case Lane and Edwards Boulevard,NaN,12/20/2019,NaN,"Millsboro, DE 19966",NaN,NaN,"Case Lane and Edwards Boulevard, Millsboro, DE..."


In [ ]:
# Print the data to a csv
flattened_data.to_csv("data/processed_data.csv", index=False)

In [ ]:
# Print the addresses to a seperate list
flattened_data["address"].to_csv("data/DSHA_addresses.csv", index=False)

At this point we transfer the address csv over to the geocoder to get the latitude and longitude of each project

In [11]:
# Read the file of geolocated addresses
geolocations = pd.read_csv("data/counts_per_tract.csv").drop_duplicates("input_address")
# Join the geolocations to the flattened dataframe

#ignore case and commas
geolocations["address_lower"] = geolocations["input_address"].str.lower().str.replace(",","")
flattened_data["address_lower"] = flattened_data["address"].str.lower().str.replace(",", "")
geolocated_data = flattened_data.merge(geolocations, on="address_lower", how="left")
# Remove lat,lot from unsuccessfully (not in the u.s.) geolocated address
geolocated_data.loc[geolocated_data["census_tract"] == "Unable To Geocode The Address", "lon"] = np.nan
geolocated_data.loc[geolocated_data["census_tract"] == "Unable To Geocode The Address", "lat"] = np.nan

In [12]:
# Convert lat,lot to Shapely points
geolocated_data = gpd.GeoDataFrame(geolocated_data, geometry=gpd.points_from_xy(geolocated_data['lon'], geolocated_data['lat'], crs="EPSG:4326"))


In [ ]:
geolocated_data

In [ ]:
# Visualize points on a map

# initialize the map and store it in a folium map object
us_map = folium.Map(location=[39.74503, -75.57203], zoom_start=14, tiles=None)

# Add background tiles
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(us_map)


# Add markers for each school
points=folium.features.GeoJson(
        geolocated_data.loc[geolocated_data["census_tract"] != "Unable To Geocode The Address"], # Full geopandas data
        control=False,
        marker = folium.CircleMarker(radius = 5, # Radius in metres
                           weight = 0, #outline weight
                           fill_color = '#d95f02', 
                           fill_opacity = 1)
        )

points.add_to(us_map)
us_map

In [13]:
# extract shape files for representative districts
rep_districts_2022 = pygris.state_legislative_districts(state="DE",house="lower", year=2022)
rep_districts_2022 = rep_districts_2022[["SLDLST","geometry"]].to_crs(4326)

rep_districts_2011 = pygris.state_legislative_districts(state="DE",house="lower", year=2011)
rep_districts_2011 = rep_districts_2011[["SLDLST","geometry"]].to_crs(4326)

Using FIPS code '10' for input 'DE'
Using FIPS code '10' for input 'DE'


In [14]:
# remove water outside of state boundary 
de = pygris.states()
de = de[de["STATEFP"]=="10"]

rep_districts_2022_trimmed = gpd.clip(rep_districts_2022, de)
rep_districts_2011_trimmed = gpd.clip(rep_districts_2011, de)

Using the default year of 2021


/var/folders/62/sjvb9hnd49j2jg4mt6p87qqm0000gn/T/ipykernel_67104/520896041.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: GEOGCS["GCS_North_American_1983",DATUM["D_North_Am ...

  rep_districts_2022_trimmed = gpd.clip(rep_districts_2022, de)
/Users/kaitlincanalichio/opt/anaconda3/envs/dsenv/lib/python3.10/site-packages/geopandas/tools/clip.py:67: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[
/var/folders/62/sjvb9hnd49j2jg4mt6p87qqm0000gn/T/ipykernel_67104/520896041.py:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to repro

In [ ]:
geolocated_data.dtypes

In [177]:
# get base map for all years 
rep_districts_2016_trimmed = rep_districts_2011_trimmed.assign(year="2016")
rep_districts_2017_trimmed = rep_districts_2011_trimmed.assign(year="2017")
rep_districts_2018_trimmed = rep_districts_2011_trimmed.assign(year="2018")
rep_districts_2019_trimmed = rep_districts_2011_trimmed.assign(year="2019")
rep_districts_2020_trimmed = rep_districts_2011_trimmed.assign(year="2020")
rep_districts_2021_trimmed = rep_districts_2011_trimmed.assign(year="2021")
rep_districts_2022_trimmed = rep_districts_2022_trimmed.assign(year="2022")

base_map = gpd.GeoDataFrame( pd.concat([rep_districts_2016_trimmed, rep_districts_2017_trimmed, rep_districts_2018_trimmed, rep_districts_2019_trimmed, rep_districts_2020_trimmed, rep_districts_2021_trimmed, rep_districts_2022_trimmed], ignore_index=True))
base_map = base_map.rename(columns={"SLDLST":"district"})
base_map["district"] = base_map["district"].astype("Int64").astype("str")
base_map.info()
# merge with data 
# base_map = base_map.merge(all_data, left_on=["SLDLST","year"], right_on=["district","year"],how="left").drop(columns=["district","geometry_y"]).rename(columns={"SLDLST":"district","geometry_x":"geometry"})

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   district  287 non-null    object  
 1   geometry  287 non-null    geometry
 2   year      287 non-null    object  
dtypes: geometry(1), object(2)
memory usage: 6.9+ KB


In [178]:
base_map

,district,geometry,year
0,40,"POLYGON ((-75.54197 38.45517, -75.54279 38.455...",2016
1,39,"POLYGON ((-75.70906 38.68455, -75.70730 38.684...",2016
2,35,"POLYGON ((-75.70190 38.68421, -75.70239 38.684...",2016
3,30,"POLYGON ((-75.56670 39.04533, -75.56690 39.043...",2016
4,34,"POLYGON ((-75.57167 39.04585, -75.57208 39.045...",2016
...,...,...,...
282,1,"POLYGON ((-75.56786 39.75602, -75.56777 39.756...",2022
283,12,"POLYGON ((-75.69456 39.80630, -75.68599 39.811...",2022
284,6,"POLYGON ((-75.55349 39.76196, -75.55304 39.762...",2022
285,7,"MULTIPOLYGON (((-75.50458 39.80275, -75.50452 ...",2022


In [179]:
# add Represenative names 
rep_names = pd.read_excel("data/Representative List.xlsx")
#convert to string type and change names
rep_names = rep_names.rename(columns={"District":"district","Name":"name"})
rep_names["district"] = rep_names["district"].astype("str") 

base_map = base_map.merge(rep_names, on="district", how="left")

In [180]:
geolocated_data.columns

Index(['PROJECT NAME & ADDRESS', 'ALLOCATION AMOUNT', 'ALLOCATION DATE',
       'Placed in Service Date', '4% or 9% Allocation',
       'Applicable Credit Rate', 'Status Active/Non', '# of Tax Credit Units',
       'Type of Property', 'County', 'Tax Credit Compliance Date',
       'Extended Use Period', 'Tax Allocation Year',
       'PROJECT NAME & ADDRESS 2', 'ALLOCATION AMOUNT 2',
       'Placed in Service Date 2', 'Applicable Credit Rate 2',
       'PROJECT NAME & ADDRESS 3', 'PROJECT NAME & ADDRESS 4',
       'PROJECT NAME & ADDRESS 5', 'address', 'address_lower', 'input_address',
       'census_tract', 'lon', 'lat', 'acc', 'geometry'],
      dtype='object')

In [194]:
# merge base map with data 

tax_data_geo = geolocated_data[["Tax Allocation Year","# of Tax Credit Units","ALLOCATION AMOUNT","geometry"]]

all_tax_geo = gpd.sjoin(base_map, tax_data_geo)

all_tax_geo_projects = gpd.sjoin(geolocated_data, base_map)

In [195]:
# make years align
all_tax_geo["Tax Allocation Year"] = all_tax_geo["Tax Allocation Year"].astype("str")
all_tax_geo = all_tax_geo[all_tax_geo["year"]==all_tax_geo["Tax Allocation Year"]]

# make years align
all_tax_geo_projects["Tax Allocation Year"] = all_tax_geo_projects["Tax Allocation Year"].astype("str")
all_tax_geo_projects = all_tax_geo_projects[all_tax_geo_projects["year"]==all_tax_geo_projects["Tax Allocation Year"]]

In [199]:
all_tax_geo_projects.rename(columns={"district":"Representative District"}).drop(columns=["index_right","input_address", "census_tract", "lon", "lat","name"]).to_file("data/DSHA_rep_districted.json")

In [169]:
# add district for correct year to geolcated data
geolocated_data["Tax Allocation Year"] = base_map["Tax Allocation Year"].astype("str")
rep_districted = geolocated_data.merge(base_map, left_on="Tax Allocation Year",right_on="year",how="left")

PROJECT NAME & ADDRESS          object
ALLOCATION AMOUNT               object
ALLOCATION DATE                 object
Placed in Service Date          object
4% or 9% Allocation             object
Applicable Credit Rate          object
Status Active/Non               object
# of Tax Credit Units           object
Type of Property                object
County                          object
Tax Credit Compliance Date      object
Extended Use Period             object
Tax Allocation Year              int64
PROJECT NAME & ADDRESS 2        object
ALLOCATION AMOUNT 2             object
Placed in Service Date 2        object
Applicable Credit Rate 2        object
PROJECT NAME & ADDRESS 3        object
PROJECT NAME & ADDRESS 4        object
PROJECT NAME & ADDRESS 5        object
address                         object
address_lower                   object
input_address                   object
census_tract                    object
lon                             object
lat                      

In [162]:
# add back base map 
base_map["year"] = base_map["year"].astype("str")
all_tax_geo = base_map.merge(all_tax_geo, on=["district","year"],how="left")
# Add funding source column
all_tax_geo["Funding Source"] = "LIHTC"

In [22]:
all_tax_geo = all_tax_geo.drop(columns=["geometry_y","name_y","index_right"]).rename(columns={"geometry_x":"geometry","name_x":"name"})

In [23]:
all_tax_geo

,district,geometry,year,name,Tax Allocation Year,# of Tax Credit Units,ALLOCATION AMOUNT,Funding Source
0,40,"POLYGON ((-75.54197 38.45517, -75.54279 38.455...",2016,Timothy D. Dukes,NaN,NaN,NaN,LIHTC
1,39,"POLYGON ((-75.70906 38.68455, -75.70730 38.684...",2016,Daniel B. Short,NaN,NaN,NaN,LIHTC
2,35,"POLYGON ((-75.70190 38.68421, -75.70239 38.684...",2016,Jesse R. Vanderwende,NaN,NaN,NaN,LIHTC
3,30,"POLYGON ((-75.56670 39.04533, -75.56690 39.043...",2016,Shannon Morris,NaN,NaN,NaN,LIHTC
4,34,"POLYGON ((-75.57167 39.04585, -75.57208 39.045...",2016,Lyndon D. Yearick,NaN,NaN,NaN,LIHTC
...,...,...,...,...,...,...,...,...
287,1,"POLYGON ((-75.56786 39.75602, -75.56777 39.756...",2022,Nnamdi O. Chukwuocha,2022,60,TBD,LIHTC
288,12,"POLYGON ((-75.69456 39.80630, -75.68599 39.811...",2022,Krista Griffith,NaN,NaN,NaN,LIHTC
289,6,"POLYGON ((-75.55349 39.76196, -75.55304 39.762...",2022,Debra Heffernan,NaN,NaN,NaN,LIHTC
290,7,"MULTIPOLYGON (((-75.50458 39.80275, -75.50452 ...",2022,Larry Lambert,NaN,NaN,NaN,LIHTC


In [24]:
# Count the number of Tax Credit Units in each district for each year and add them to the aggregated dataframe
all_tax_geo['# of Tax Credit Units'] = all_tax_geo['# of Tax Credit Units'].astype("Int64")
aggregated_data = all_tax_geo.groupby(["district", "Tax Allocation Year"]).sum()["# of Tax Credit Units"].reset_index()
aggregated_data.to_csv("data/Tax_Credit_Units_per_Representative_District.csv", index=False)





/var/folders/62/sjvb9hnd49j2jg4mt6p87qqm0000gn/T/ipykernel_67104/819291913.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_data = all_tax_geo.groupby(["district", "Tax Allocation Year"]).sum()["# of Tax Credit Units"].reset_index()


In [25]:
# Count the Allocation Amount in each district for each year and add them to the aggregated dataframe
data_noTBD = all_tax_geo.loc[all_tax_geo['ALLOCATION AMOUNT'] != "TBD"]
data_noTBD["ALLOCATION AMOUNT"] = data_noTBD['ALLOCATION AMOUNT'].astype("Int64")
aggregated_data = aggregated_data.merge(data_noTBD.groupby(["district", "Tax Allocation Year"]).sum()["ALLOCATION AMOUNT"].reset_index(), how="outer", on=["district", "Tax Allocation Year"])

/var/folders/62/sjvb9hnd49j2jg4mt6p87qqm0000gn/T/ipykernel_67104/415686677.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_noTBD["ALLOCATION AMOUNT"] = data_noTBD['ALLOCATION AMOUNT'].astype("Int64")
/var/folders/62/sjvb9hnd49j2jg4mt6p87qqm0000gn/T/ipykernel_67104/415686677.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_data = aggregated_data.merge(data_noTBD.groupby(["district", "Tax Allocation Year"]).sum()["ALLOCATION AMOUNT"].reset_index(), how="outer", on=["district", "Tax Allocation Year"])


In [26]:
aggregated_data

,district,Tax Allocation Year,# of Tax Credit Units,ALLOCATION AMOUNT
0,1,2017,53,778622
1,1,2022,60,<NA>
2,2,2019,59,1056240
3,2,2020,50,621046
4,2,2021,50,1000000
5,2,2022,51,1000000
6,24,2017,60,774946
7,28,2021,54,<NA>
8,3,2016,51,913337
9,3,2018,77,1122778


In [27]:
# get the population per district for each year 
population = pd.read_csv("data/rep_district_population.csv")
population["District"] = population["District"].astype("str")
population["year"] = population["year"].astype("str")


In [28]:
# calculations
# add population data 
all_tax_geo["Tax Allocation Year"] = all_tax_geo["Tax Allocation Year"].astype("str")

aggregated_data = aggregated_data.merge(population, how='left', left_on=["district","Tax Allocation Year"], right_on=["District","year"])

aggregated_data["Average Allocation per 100 Persons"] = (aggregated_data["ALLOCATION AMOUNT"] / aggregated_data["population"]) *100
aggregated_data["Average Population per Tax Credit Unit"] = aggregated_data["population"] / aggregated_data["# of Tax Credit Units"]
aggregated_data["Average Allocation per Tax Credit Unit"] = aggregated_data["ALLOCATION AMOUNT"] / aggregated_data["# of Tax Credit Units"]



In [29]:
aggregated_data

,district,Tax Allocation Year,# of Tax Credit Units,ALLOCATION AMOUNT,year,District,population,Average Allocation per 100 Persons,Average Population per Tax Credit Unit,Average Allocation per Tax Credit Unit
0,1,2017,53,778622,2017,1,22303,3491.108819,420.811321,14690.981132
1,1,2022,60,<NA>,2022,1,22262,<NA>,371.033333,<NA>
2,2,2019,59,1056240,2019,2,21794,4846.471506,369.389831,17902.372881
3,2,2020,50,621046,2020,2,21782,2851.189055,435.64,12420.92
4,2,2021,50,1000000,2021,2,22198,4504.910352,443.96,20000.0
5,2,2022,51,1000000,2022,2,22198,4504.910352,435.254902,19607.843137
6,24,2017,60,774946,2017,24,23230,3335.970728,387.166667,12915.766667
7,28,2021,54,<NA>,2021,28,23386,<NA>,433.074074,<NA>
8,3,2016,51,913337,2016,3,21271,4293.813173,417.078431,17908.568627
9,3,2018,77,1122778,2018,3,19408,5785.129843,252.051948,14581.532468


In [31]:
all_data

,district,geometry,year,name,District,population,Funding Source
0,40,"POLYGON ((-75.54197 38.45517, -75.54279 38.455...",2016,Timothy D. Dukes,40,22131,LIHTC
1,39,"POLYGON ((-75.70906 38.68455, -75.70730 38.684...",2016,Daniel B. Short,39,23198,LIHTC
2,35,"POLYGON ((-75.70190 38.68421, -75.70239 38.684...",2016,Jesse R. Vanderwende,35,22522,LIHTC
3,30,"POLYGON ((-75.56670 39.04533, -75.56690 39.043...",2016,Shannon Morris,30,21840,LIHTC
4,34,"POLYGON ((-75.57167 39.04585, -75.57208 39.045...",2016,Lyndon D. Yearick,34,23318,LIHTC
...,...,...,...,...,...,...,...
282,1,"POLYGON ((-75.56786 39.75602, -75.56777 39.756...",2022,Nnamdi O. Chukwuocha,1,22262,LIHTC
283,12,"POLYGON ((-75.69456 39.80630, -75.68599 39.811...",2022,Krista Griffith,12,23385,LIHTC
284,6,"POLYGON ((-75.55349 39.76196, -75.55304 39.762...",2022,Debra Heffernan,6,21968,LIHTC
285,7,"MULTIPOLYGON (((-75.50458 39.80275, -75.50452 ...",2022,Larry Lambert,7,22773,LIHTC


In [30]:
# merge with base_map to get the names of the Representatives 
all_data = base_map.merge(population, how="left", left_on=["district","year"], right_on=["District","year"])
all_data["Funding Source"] = "LIHTC"

In [34]:
all_data

,district,geometry,year,name,District,population,Funding Source
0,40,"POLYGON ((-75.54197 38.45517, -75.54279 38.455...",2016,Timothy D. Dukes,40,22131,LIHTC
1,39,"POLYGON ((-75.70906 38.68455, -75.70730 38.684...",2016,Daniel B. Short,39,23198,LIHTC
2,35,"POLYGON ((-75.70190 38.68421, -75.70239 38.684...",2016,Jesse R. Vanderwende,35,22522,LIHTC
3,30,"POLYGON ((-75.56670 39.04533, -75.56690 39.043...",2016,Shannon Morris,30,21840,LIHTC
4,34,"POLYGON ((-75.57167 39.04585, -75.57208 39.045...",2016,Lyndon D. Yearick,34,23318,LIHTC
...,...,...,...,...,...,...,...
282,1,"POLYGON ((-75.56786 39.75602, -75.56777 39.756...",2022,Nnamdi O. Chukwuocha,1,22262,LIHTC
283,12,"POLYGON ((-75.69456 39.80630, -75.68599 39.811...",2022,Krista Griffith,12,23385,LIHTC
284,6,"POLYGON ((-75.55349 39.76196, -75.55304 39.762...",2022,Debra Heffernan,6,21968,LIHTC
285,7,"MULTIPOLYGON (((-75.50458 39.80275, -75.50452 ...",2022,Larry Lambert,7,22773,LIHTC


In [40]:
all_data_agg = all_data.merge(aggregated_data, on=["district","year"],how="left")

In [41]:
all_data_agg = all_data_agg.drop(columns=["District_x","District_y","population_y"]).rename(columns={"population_x":"population"})
all_data_agg = all_data_agg.replace("<NA>",None)
all_data_agg = all_data_agg.fillna(0)

In [43]:
aggregated_data

,district,Tax Allocation Year,# of Tax Credit Units,ALLOCATION AMOUNT,year,District,population,Average Allocation per 100 Persons,Average Population per Tax Credit Unit,Average Allocation per Tax Credit Unit
0,1,2017,53,778622,2017,1,22303,3491.108819,420.811321,14690.981132
1,1,2022,60,<NA>,2022,1,22262,<NA>,371.033333,<NA>
2,2,2019,59,1056240,2019,2,21794,4846.471506,369.389831,17902.372881
3,2,2020,50,621046,2020,2,21782,2851.189055,435.64,12420.92
4,2,2021,50,1000000,2021,2,22198,4504.910352,443.96,20000.0
5,2,2022,51,1000000,2022,2,22198,4504.910352,435.254902,19607.843137
6,24,2017,60,774946,2017,24,23230,3335.970728,387.166667,12915.766667
7,28,2021,54,<NA>,2021,28,23386,<NA>,433.074074,<NA>
8,3,2016,51,913337,2016,3,21271,4293.813173,417.078431,17908.568627
9,3,2018,77,1122778,2018,3,19408,5785.129843,252.051948,14581.532468


In [128]:
all_data_agg

,district,geometry,year,name,population,Funding Source,Tax Allocation Year,# of Tax Credit Units,ALLOCATION AMOUNT,Average Allocation per 100 Persons,Average Population per Tax Credit Unit,Average Allocation per Tax Credit Unit
0,40,"POLYGON ((-75.54197 38.45517, -75.54279 38.455...",2016,Timothy D. Dukes,22131,LIHTC,0,0,0,0.0,0.0,0.0
1,39,"POLYGON ((-75.70906 38.68455, -75.70730 38.684...",2016,Daniel B. Short,23198,LIHTC,0,0,0,0.0,0.0,0.0
2,35,"POLYGON ((-75.70190 38.68421, -75.70239 38.684...",2016,Jesse R. Vanderwende,22522,LIHTC,0,0,0,0.0,0.0,0.0
3,30,"POLYGON ((-75.56670 39.04533, -75.56690 39.043...",2016,Shannon Morris,21840,LIHTC,0,0,0,0.0,0.0,0.0
4,34,"POLYGON ((-75.57167 39.04585, -75.57208 39.045...",2016,Lyndon D. Yearick,23318,LIHTC,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
282,1,"POLYGON ((-75.56786 39.75602, -75.56777 39.756...",2022,Nnamdi O. Chukwuocha,22262,LIHTC,2022,60,0,0.0,371.033333,0.0
283,12,"POLYGON ((-75.69456 39.80630, -75.68599 39.811...",2022,Krista Griffith,23385,LIHTC,0,0,0,0.0,0.0,0.0
284,6,"POLYGON ((-75.55349 39.76196, -75.55304 39.762...",2022,Debra Heffernan,21968,LIHTC,0,0,0,0.0,0.0,0.0
285,7,"MULTIPOLYGON (((-75.50458 39.80275, -75.50452 ...",2022,Larry Lambert,22773,LIHTC,0,0,0,0.0,0.0,0.0


In [112]:
long_data[(long_data["variable"]=="# of Tax Credit Units") & (long_data["value"]!=0)]

,district,name,Funding Source,Tax Allocation Year,variable,value
10,41,Richard G. Collins,LIHTC,2016,# of Tax Credit Units,38
12,37,Ruth Briggs King,LIHTC,2016,# of Tax Credit Units,32
31,3,Sherry Dorsey Walker,LIHTC,2016,# of Tax Credit Units,51
33,4,Jeff Hilovsky,LIHTC,2016,# of Tax Credit Units,72
43,35,Jesse R. Vanderwende,LIHTC,2017,# of Tax Credit Units,34
46,31,Sean M. Lynn,LIHTC,2017,# of Tax Credit Units,100
51,41,Richard G. Collins,LIHTC,2017,# of Tax Credit Units,56
65,5,Kendra Johnson,LIHTC,2017,# of Tax Credit Units,120
67,24,Edward S. Osienski,LIHTC,2017,# of Tax Credit Units,60
73,1,Nnamdi O. Chukwuocha,LIHTC,2017,# of Tax Credit Units,53


In [144]:
# Melt the wide form data into long form data, grouping by district, representative name, funding source, and year
long_data = pd.melt(all_data_agg, id_vars=["district", "name", "Funding Source", "Tax Allocation Year"], value_vars=["# of Tax Credit Units", "ALLOCATION AMOUNT", "population", "Average Allocation per 100 Persons", "Average Population per Tax Credit Unit", "Average Allocation per Tax Credit Unit"])

# Calculate the average of each variable across all districts and years

averages = long_data.loc[long_data["Tax Allocation Year"]!=0].groupby("variable").mean(numeric_only=False).reset_index()
averages["Funding Source"] = "LIHTC"
averages["name"] = np.nan
averages["district"] = "District Average"
averages["Tax Allocation Year"] = "All Time"

# Calculate the average of each variable across all districts in each year
yearly_averages = long_data.loc[long_data["Tax Allocation Year"]!=0].groupby(["variable", "Tax Allocation Year"]).mean(numeric_only=False).reset_index()
yearly_averages["Funding Source"] = "LIHTC"
yearly_averages["name"] = np.nan
yearly_averages["district"] = "District Average"

# Add the averages to the long form data
long_data = pd.concat([long_data,averages, yearly_averages]).drop_duplicates().reset_index(drop=True)
# Fill missing values with 0 for processing
long_data.fillna(0).drop_duplicates().to_csv("data/long_tax_data_reps.csv", index=False)

/var/folders/62/sjvb9hnd49j2jg4mt6p87qqm0000gn/T/ipykernel_67104/796110383.py:6: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  averages = long_data.loc[long_data["Tax Allocation Year"]!=0].groupby("variable").mean(numeric_only=False).reset_index()
/var/folders/62/sjvb9hnd49j2jg4mt6p87qqm0000gn/T/ipykernel_67104/796110383.py:13: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  yearly_averages = long_data.loc[long_data["Tax Allocation Year"]!=0].groupby(["variable", "Tax Allocation Year"]).mean(numeric_only=False).reset_index()


In [143]:
print(long_data.to_string())

             district                   name Funding Source Tax Allocation Year                                variable          value
0                  40       Timothy D. Dukes          LIHTC                   0                   # of Tax Credit Units              0
1                  39        Daniel B. Short          LIHTC                   0                   # of Tax Credit Units              0
2                  35   Jesse R. Vanderwende          LIHTC                   0                   # of Tax Credit Units              0
3                  30         Shannon Morris          LIHTC                   0                   # of Tax Credit Units              0
4                  34      Lyndon D. Yearick          LIHTC                   0                   # of Tax Credit Units              0
5                  31           Sean M. Lynn          LIHTC                   0                   # of Tax Credit Units              0
6                   9        Kevin S Hensley          L

In [ ]:
rep_districts_2011_trimmed

In [ ]:
# Attach representative districts to wide form aggregated data with appropriate year 
aggregated_data2011 = all_data_agg[all_data_agg['year'] != "2022"] 
#aggregated_data2011 = rep_districts_2011_trimmed.merge(aggregated_data2011, right_on="district", left_on="SLDLST", how="outer")

aggregated_data2022 = all_data_agg[all_data_agg['year'] == "2022"] 
#aggregated_data2022 = rep_districts_2022_trimmed.merge(aggregated_data2022, right_on="district", left_on="SLDLST", how="outer")


In [ ]:
aggregated_data2022

In [ ]:
# add sum over all years to 2022 dataframe
years_sum = all_data_agg.groupby("district")[["# of Tax Credit Units", "ALLOCATION AMOUNT", "population", "Average Allocation per 100 Persons", "Average Population per Tax Credit Unit", "Average Allocation per Tax Credit Unit"]].apply(sum).reset_index()
years_sum["year"] = "Sum over All Time"
years_sum["Tax Allocation Year"] = "Sum over All Time"
years_sum["Funding Source"] = "Sum over All Time"


In [ ]:
base_map22 = base_map[base_map["year"]=="2022"]

In [ ]:
years_sum

In [ ]:
# get rep names and geometry then merge with years sum
years_sum = years_sum.merge(base_map22, on="district",how="left")
years_sum = years_sum.drop(columns="year_y").rename(columns={"year_x":"year"})
# reorder columns to concat
years_sum.loc[:,["district","geometry","year","name","population","Funding Source","Tax Allocation Year","# of Tax Credit Units","ALLOCATION AMOUNT","Average Allocation per 100 Persons","Average Population per Tax Credit Unit","Average Allocation per Tax Credit Unit"]]

final2022 = gpd.GeoDataFrame(pd.concat([aggregated_data2022, years_sum],axis=0,ignore_index=True))

In [ ]:
aggregated_data2011

In [ ]:
final2022[["# of Tax Credit Units", "ALLOCATION AMOUNT","Population"]]=final2022[["# of Tax Credit Units", "ALLOCATION AMOUNT","Population"]].astype("Int64").astype("str")

final2011[["Tax Allocation Year","# of Tax Credit Units", "ALLOCATION AMOUNT","Population"]]=final2011[["Tax Allocation Year","# of Tax Credit Units", "ALLOCATION AMOUNT","Population"]].astype("Int64").astype("str")


In [ ]:
final2022[["# of Tax Credit Units","ALLOCATION AMOUNT","Average Allocation per 100 Persons","Average Population per Tax Credit Unit","Average Allocation per Tax Credit Unit"]]=final2022[["# of Tax Credit Units","ALLOCATION AMOUNT","Average Allocation per 100 Persons","Average Population per Tax Credit Unit","Average Allocation per Tax Credit Unit"]].astype(float)
aggregated_data2011[["# of Tax Credit Units","ALLOCATION AMOUNT","Average Allocation per 100 Persons","Average Population per Tax Credit Unit","Average Allocation per Tax Credit Unit"]]=aggregated_data2011[["# of Tax Credit Units","ALLOCATION AMOUNT","Average Allocation per 100 Persons","Average Population per Tax Credit Unit","Average Allocation per Tax Credit Unit"]].astype(float)

In [ ]:
# save as GeoJSON

gpd.GeoDataFrame(aggregated_data2011).to_file("data/reps_aggregated_with_geo2011.geojson", driver="GeoJSON")
gpd.GeoDataFrame(final2022).to_file("data/reps_aggregated_with_geo2022.geojson", driver="GeoJSON")

In [ ]:
final2022[final2022["district"]=="2"]

In [ ]:
# Attach senate districts to wide form aggregated data
aggregated_data["district"] = pd.to_numeric(aggregated_data["district"], errors="coerce")
aggregated_data = aggregated_data.merge(trim, on="district", how="outer")
# Print the wide form data for valid districts to a geojson
gpd.GeoDataFrame(aggregated_data.loc[~aggregated_data["district"].isna()]).to_file("data/aggregated_with_geo.geojson", driver="GeoJSON")
# Print the wide form data without geoometry to a csv
aggregated_data.drop(columns="geometry").to_csv("data/aggregated_data_with_na.csv", index=False)

Figure out how senate districts overlap with senate districts


NOTE: PyPDF2 needs to be added to the environment and imported to run this code

In [ ]:
# Download census block groups
blocks = requests.get("https://www2.census.gov/geo/tiger/GENZ2022/shp/cb_2022_10_bg_500k.zip")
blocks = gpd.read_file(BytesIO(blocks.content))
blocks

In [ ]:
lines = []

# creating a pdf file object
with open('data/CensusBlockBreakdownbySenateDistrict.pdf', 'rb') as pdfFileObj:
    # creating a pdf reader object
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    # printing number of pages in pdf file
    print(pdfReader.numPages)

    # Iterate over each page
    for pageNum in range(pdfReader.numPages):
        # creating a page object
        pageObj = pdfReader.getPage(pageNum)
        
        # extracting text from page
        pageText = pageObj.extractText()
        
        # extract lines from each page
        pageLines = pageText.split("\n")
        
        for line in pageLines:
            lines.append(line.split(' '))

# Pull the columns out and separate columns that were incorrectly joined
columns = lines[0]
columns[1] = columns[0][5:] + columns[1]
columns[0] = columns[0][:5]
columns.insert(2, columns[2][:6])
columns[3] = columns[3][6:]
data = [x for x in lines if x[0].isdigit()]
for row in data:
    row.insert(1, row[0][2:])
    row[0] = row[0][:2]
    row.insert(2, row[2][:5])
    row[3] = row[3][5:]
    
senate_census_map = pd.DataFrame(data, columns=columns)
senate_census_map

In [ ]:
# Crosstabulate the connections between senate districts and census block groups
crosstab = pd.crosstab(senate_census_map["Proposed2022_SD"], senate_census_map["BlockGroup"])
# Get a list of census block groups that overlap with each senate district
senate_per_bg = crosstab.apply(lambda x : x[x != 0].index.values)
senate_per_bg.loc[senate_per_bg.apply(len) != 1]




In [ ]:
senate_per_bg2 = crosstab.apply(lambda x : x[x != 0].values)
senate_per_bg2.loc[senate_per_bg2.apply(len) != 1]

In [ ]:
scbg = pd.concat([senate_per_bg, senate_per_bg2], axis=1)
scbg.loc[(scbg.apply(lambda x : x.apply(len)) != 1).all(axis=1)].values

In [ ]:
scbg.columns = ["Senate Districts", "Blocks per District"]
scbg.to_csv("data/Senate_to_Block_Groups.csv")